In [1]:
import torch
from model_bruno import *
from torchvision import datasets, transforms
from time import time
from tqdm import tqdm 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
model = torch.load('trained_models/model100epochsLRe4.pt')
model = model.to(device)
model.eval() 

AttModel(
  (classifier): Linear(in_features=512, out_features=10, bias=True)
  (act_class): Softmax(dim=1)
  (backbone): AnchorBackbone(
    (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2))
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    (conv3): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2))
    (anchor0): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
    (anchor1): Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1))
    (anchor2): Conv2d(512, 1024, kernel_size=(3, 1), stride=(1, 1))
    (anchor3): Conv2d(512, 1024, kernel_size=(1, 3), stride=(1, 1))
    (conv_act): ReLU()
  )
  (attention_head): AttentionHead(
    (W1): Linear(in_features=1024, out_features=512, bias=True)
    (Wa): Linear(in_features=256, out_features=1, bias=True)
    (U): Linear(in_features=512, out_features=256, bias=True)
    (V): Linear(in_features=512, out_features=256, bias=True)
    (act_W1): ReLU()
    (act_Wa): ReLU()
    (act_U): Sigmoid()
    (act_V): Tanh()
 

In [3]:
transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip()      
])

STL10_set = datasets.STL10(root='C:/Users/Bruno/OneDrive/Documents/Datasets/STL10', split='test', transform=transformation, download=False)
STL10_loader = torch.utils.data.DataLoader(STL10_set, batch_size=1, shuffle=True,  pin_memory=False)

In [4]:
tic = time()
avg_acc = 0
for x,y in tqdm(STL10_loader):
    x = x.to(device)
    y = y.to(device)

    pred, _, _ = model(x)                
    avg_acc += (y == pred.argmax(1)).float().mean() / len(STL10_loader)   

toc = time()

print(f'{toc-tic:.0f}s - Avg acc: {avg_acc}')

100%|██████████| 8000/8000 [2:45:21<00:00,  1.24s/it]  

9922s - Avg acc: 0.6431012153625488
